<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/thermodynamics/phaseenvelopesofCO2richmixtures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
%%capture
!pip install neqsim

#Phase envelopes of CO2 rich mixtures
In the following examples we will use the UMR-PRU EoS for calculation of phase behaviour of CO2 rich mixtures.


##Phase curve of pure CO2
